# Machine Learning - Doc2Vec Basis

## Import Libraries

In [1]:
import logging
import os
import gensim
import smart_open
import collections
import random
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Data Import

In [2]:
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [4]:
print(train_corpus[:2])

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

In [5]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

In [6]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40) #word count = 2 in order to discard words with very few occurrences
model.build_vocab(train_corpus)
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")

## Essentially, the vocabulary is a list (accessible via model.wv.index_to_key) of all of the unique words extracted from the training corpus.
## Additional attributes for each word are available using the model.wv.get_vecattr() method

2024-03-28 16:17:12,146 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>', 'datetime': '2024-03-28T16:17:12.146973', 'gensim': '4.3.2', 'python': '3.9.6 (default, Feb  3 2024, 15:58:27) \n[Clang 15.0.0 (clang-1500.3.9.4)]', 'platform': 'macOS-14.3.1-arm64-arm-64bit', 'event': 'created'}
2024-03-28 16:17:12,147 : INFO : collecting all words and their counts
2024-03-28 16:17:12,147 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2024-03-28 16:17:12,152 : INFO : collected 6981 word types and 300 unique tags from a corpus of 300 examples and 58152 words
2024-03-28 16:17:12,152 : INFO : Creating a fresh vocabulary
2024-03-28 16:17:12,160 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 3955 unique words (56.65% of original 6981, drops 3026)', 'datetime': '2024-03-28T16:17:12.160132', 'gensim': '4.3.2', 'python': '3.9.6 (default, Feb  3 2024, 15:58:27) \n[Clang 15.0.0 (clang-1500.3.9.4)]', 'platfor

Word 'penalty' appeared 4 times in the training corpus.


## Model Training

In [7]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2024-03-28 16:17:12,187 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 3955 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2024-03-28T16:17:12.187386', 'gensim': '4.3.2', 'python': '3.9.6 (default, Feb  3 2024, 15:58:27) \n[Clang 15.0.0 (clang-1500.3.9.4)]', 'platform': 'macOS-14.3.1-arm64-arm-64bit', 'event': 'train'}
2024-03-28 16:17:12,205 : INFO : EPOCH 0: training on 58152 raw words (42735 effective words) took 0.0s, 2655117 effective words/s
2024-03-28 16:17:12,221 : INFO : EPOCH 1: training on 58152 raw words (42654 effective words) took 0.0s, 2809149 effective words/s
2024-03-28 16:17:12,237 : INFO : EPOCH 2: training on 58152 raw words (42809 effective words) took 0.0s, 2940036 effective words/s
2024-03-28 16:17:12,253 : INFO : EPOCH 3: training on 58152 raw words (42672 effective words) took 0.0s, 2865102 effective words/s
2024-03-28 16:17:12,269 : INFO : EPOCH 4: training on 58152 ra

In [8]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

# Now, we can use the trained model to infer a vector for any piece of text by passing a list of words to the model.infer_vector function.
# This vector can then be compared with other vectors via cosine similarity.
# Note that infer_vector() does not take a string, but rather a list of string tokens.
# tokenized the same way as the words property of original training document objects.

[-2.28579804e-01 -1.93387255e-01 -1.91703320e-01  2.08844453e-01
 -6.55727312e-02 -2.23161653e-02  3.55680250e-02  1.42806068e-01
 -2.45812774e-01 -1.68189570e-01  1.72876731e-01  9.75875705e-02
 -1.28075397e-02 -2.41612666e-04 -1.42846361e-01 -1.09524399e-01
  6.15948327e-02  1.03017479e-01  7.86293373e-02 -1.76768988e-01
 -1.16573326e-01  4.21345644e-02  1.47841707e-01 -4.29139845e-03
 -9.70738456e-02  3.80495889e-03 -2.33180165e-01  1.89270023e-02
 -1.45805091e-01 -2.27556611e-05  4.73222017e-01  4.51947562e-02
  1.86542377e-01  1.99952617e-01  2.04876631e-01  2.24792957e-01
 -9.81401578e-02 -1.50636137e-01 -1.71758890e-01 -3.71105634e-02
 -7.57913920e-04  5.63901290e-02  4.68456447e-02 -7.22627342e-02
  1.40534073e-01 -4.47633639e-02 -1.12327784e-01 -8.34705755e-02
  1.09989546e-01  7.46573955e-02]


In [9]:
## The expectation is that we’ve likely overfit our model (i.e., all of the ranks will be less than 2)

ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [10]:
counter = collections.Counter(ranks)
print(counter)
# Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document.
# Checking the inferred-vector against a training-vector is a sort of ‘sanity check’ as to whether the model is behaving in a usefully consistent manner, though not a real ‘accuracy’ value.

Counter({0: 292, 1: 8})


In [11]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [12]:
# Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (171): «drug education campaigns appear to be paying dividends with new figures showing per cent drop in drug related deaths last year according to the australian bureau of statistics people died from drug related causes in the year that figure is substantial drop from when australians died of drug related causes across the states and territories new south wales recorded the biggest decrease the bureau david payne attributes the decline of drug deaths to the heroin drought in some parts of the country better equipped ambulances and emergency wards and above all effective federal and state drug education campaigns they have put lot of money into the program there has been fall and while you can discern trend from that the figures are going in the right way right direction mr payne said»

Similar Document (123, 0.721383810043335): «new report has revealed there are fewer young people using homeless services than widely thought the australian institute of health and welfare

## Model Testing

In [13]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (11): «intelligence cannot say conclusively that saddam hussein has weapons of mass destruction an information gap that is complicating white house efforts to build support for an attack on saddam iraqi regime the cia has advised top administration officials to assume that iraq has some weapons of mass destruction but the agency has not given president bush smoking gun according to intelligence and administration officials»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>:

MOST (75, 0.7714795470237732): «us president george bush has marked the th day of the campaign against terrorism by calling on his allies to freeze the assets of two non us organisations suspected of supporting terrorism one of the groups is based in kashmir the other is alleged to have helped al qaeda develop nuclear weapons president bush says former scientist at pakistan atomic program had established group called utn after assisting osama bin laden network develop nuclear bo